In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [162]:
import keras
import nltk
import numpy
nltk.download('punkt')
from nltk import punkt
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize 
from keras import backend
from keras.models import Sequential
from keras.layers import Activation, Conv1D
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
#importing files as text

train_path = "train.txt"
test_path = "test.txt"

#in your case insert your path for train_path and test_path


f = open(train_path, 'r')
train_text = [j.split('\n') for j in f.read().split('\n\n')]
for i in range (len(train_text)):
  for j in range (len(train_text[i])):
    train_text[i][j] = train_text[i][j].split('\t')
    train_text[i][j][0] = train_text[i][j][0].lower()
f.close()

f = open(test_path, 'r')
test_text = [j.split('\n') for j in f.read().split('\n\n')]
for i in range (len(test_text)):
  for j in range (len(test_text[i])):
    test_text[i][j].strip()
    test_text[i][j] = test_text[i][j].split('\t')
    test_text[i][j][0] = test_text[i][j][0].lower()
    # test_text[i][j][1].lower()
f.close()

In [0]:
#converting the data in more meaningful way

trainX = []
trainY = []
testX = []
testY = []

for i in range (len(train_text)):
    if len(train_text[i][0]) == 3:
      temp_train = []
      if train_text[i][0][2].lower() == "positive":
        trainY.append(1)
      elif train_text[i][0][2].lower() == "neutral":
        trainY.append(0)
      else:
        trainY.append(-1)
      for j in range (1,len(train_text[i])):
        if len(train_text[i][j]) == 2:
        # if train_text[i][j][1] != 'O' and len(train_text[i][j]) == 2:
          temp_train.append(train_text[i][j])
      trainX.append(temp_train)
    # else:
    #   print('wrong')

for i in range (len(test_text)):
  if len(test_text[i][0]) == 3:
    temp_test = []
    if test_text[i][0][2].lower() == "positive":
      testY.append(1)
    elif test_text[i][0][2].lower() == "neutral":
      testY.append(0)
    else:
      testY.append(-1)
    for j in range (1,len(test_text[i])):
      if len(test_text[i][j]) == 2:
      # if test_text[i][j][1] != 'O' and len(test_text[i][j]) == 2:
        temp_test.append(test_text[i][j])
    testX.append(temp_test)
  # else:
  #   print('wrong')

In [0]:
dic = dict()
max_length = max([len(i) for i in trainX])
count = 0
for i in range (len(trainX)):
  for j in range (len(trainX[i])):
    if trainX[i][j][0] not in dic:
      count += 1
      dic[trainX[i][j][0]] = count
      trainX[i][j] = count
    else:
      trainX[i][j] = dic[trainX[i][j][0]]

for i in range (len(testX)):
  for j in range (len(testX[i])):
    if testX[i][j][0] not in dic:
      count += 1
      dic[testX[i][j][0]] = count
      testX[i][j] = count
    else:
      testX[i][j] = dic[testX[i][j][0]]

# print(len(dic))
# print(max_length)
# print(count)

In [0]:
for i in range (len(trainX)):
  trainX[i] = trainX[i] + [0]*(max_length-len(trainX[i]))

for i in range (len(testX)):
  testX[i] = testX[i] + [0]*(max_length-len(testX[i]))

trainX = numpy.array(trainX)
testX = numpy.array(testX)

In [0]:
model = Sequential([
                    
                    
                    Dense(5000, input_shape=(max_length,), activation='linear'),
                    Dense(1000, activation='sigmoid'),
                    Dense(200, activation='tanh'),
                    Dense(50, activation='tanh'),
                    Dense(1, activation='tanh')
])

In [158]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_105 (Dense)            (None, 5000)              285000    
_________________________________________________________________
dense_106 (Dense)            (None, 1000)              5001000   
_________________________________________________________________
dense_107 (Dense)            (None, 200)               200200    
_________________________________________________________________
dense_108 (Dense)            (None, 50)                10050     
_________________________________________________________________
dense_109 (Dense)            (None, 1)                 51        
Total params: 5,496,301
Trainable params: 5,496,301
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(Adam(lr=0.0001), loss='mean_squared_error', metrics=['accuracy'])

In [160]:
model.fit(trainX, trainY, batch_size=10, epochs=5, validation_split=0)

Epoch 1/5
15131/15131 [==============================] - 11s 716us/step - loss: 0.6417 - acc: 0.3724
Epoch 2/5
15131/15131 [==============================] - 9s 572us/step - loss: 0.6178 - acc: 0.3733
Epoch 3/5
15131/15131 [==============================] - 9s 571us/step - loss: 0.6146 - acc: 0.3727
Epoch 4/5
15131/15131 [==============================] - 9s 583us/step - loss: 0.6141 - acc: 0.3723
Epoch 5/5
15131/15131 [==============================] - 9s 568us/step - loss: 0.6142 - acc: 0.3729


In [161]:
model.evaluate(x=testX, y=testY, batch_size=None, verbose=1, sample_weight=None, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False)

1869/1869 [==============================] - 1s 375us/step


[0.5877378519490928, 0.4034242911046045]